In [36]:
# Import needed packages
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
from net import *

In [37]:
# CIFAR 10 classes in order
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [38]:

""" Instantiate model, this downloads tje 4.7 mb  squzzene the first time it is called.
To use with your own model, re-define your trained networks ad load weights as below
checkpoint = torch.load("pathtosavemodel")
model = SimpleNet(num_classes=10)
model.load_state_dict(checkpoint)
model.eval()
"""

checkpoint = torch.load("models/cifar10_1layers_32channels_epoch9.model")
model = SimpleNet(num_classes=10)
model.load_state_dict(checkpoint)
model.eval()

SimpleNet(
  (net): Sequential(
    (0): Unit(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (1): MaxPool2d(kernel_size=32, stride=32, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=32, out_features=10, bias=True)
)

In [39]:
# Predicts a given image
def predict_image(image_tensor):
    print("Prediction in progress")

    if torch.cuda.is_available():
        image_tensor.cuda()

    # Turn the input into a Variable
    input = Variable(image_tensor)

    # Predict the class of the image
    output = model(input)
    print(output)
    index = output.data.numpy().argmax()

    return index

In [40]:
# Define transformations for the image, right now just converting to tensor
transformation = transforms.Compose([
    transforms.ToTensor()
])


# Load the test set, note that train is set to False
test_set = CIFAR10(root="data", train=False, transform=transformation, download=True)

# Create a temp dataset loader just to load images, with batch size 1
test_loader = DataLoader(test_set, batch_size=1,shuffle=False, num_workers=1)

for i, (image, label) in enumerate(test_loader):
    
    index = predict_image(image)
    print("Predicted Class {0}: Actual Class {1}".format(index, label.data[0]))

    if i > 10:
        print("Exiting")
        break

Files already downloaded and verified
Prediction in progress
tensor([[ 0.8905,  0.2973,  0.7494,  1.5389, -0.2012,  1.9713, -1.7139, -1.1065,
          1.1292, -1.4401]], grad_fn=<AddmmBackward>)
Predicted Class 5: Actual Class 3
Prediction in progress
tensor([[ 0.5693,  3.0169, -1.5151, -0.1009, -3.7500,  0.9151, -4.5564, -0.1970,
          2.8849,  2.6096]], grad_fn=<AddmmBackward>)
Predicted Class 1: Actual Class 8
Prediction in progress
tensor([[ 2.1743,  0.4126, -0.7805, -0.3309, -1.6924, -0.4971, -2.2978, -1.8465,
          4.5636,  1.3130]], grad_fn=<AddmmBackward>)
Predicted Class 8: Actual Class 8
Prediction in progress
tensor([[ 1.9447,  1.5132, -0.0418, -0.4418, -1.6817,  0.0707, -2.2343, -0.8734,
          3.1662,  0.8963]], grad_fn=<AddmmBackward>)
Predicted Class 8: Actual Class 0
Prediction in progress
tensor([[-0.7552, -1.5844,  1.9645,  0.9720,  3.1564,  1.5989,  3.3489,  1.5314,
         -1.6788, -2.6464]], grad_fn=<AddmmBackward>)
Predicted Class 6: Actual Class 6
Pr